# Chapter 9: Evaluation Metrics for LLMs

**Portfolio Project: Building LLMs from Scratch on AWS** 📊

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/yourusername/llm-from-scratch-aws/blob/main/09_Evaluation_Metrics.ipynb)

---

## 📋 Chapter Overview

Comprehensive evaluation metrics for language models:
- **ROUGE**: Summarization quality
- **BLEU**: Translation quality
- **Perplexity**: Language modeling capability
- **METEOR**: Advanced translation metric
- **BERTScore**: Semantic similarity
- **Human Evaluation**: Guidelines and tools

**Learning Objectives:**
✅ Implement evaluation metrics from scratch  
✅ Understand metric strengths and limitations  
✅ Compare model outputs systematically  
✅ Production monitoring strategies  

**AWS Services:** CloudWatch, S3 for result storage  
**Estimated Cost:** $1-3

---


## 🔧 Setup

### Cell Purpose: Install dependencies


In [ ]:
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    !pip install -q torch matplotlib numpy nltk rouge-score sacrebleu
    
import torch
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter
import math
import re

# Try to import evaluation libraries
try:
    from rouge_score import rouge_scorer
    from sacrebleu.metrics import BLEU
    EVAL_LIBS_AVAILABLE = True
except ImportError:
    EVAL_LIBS_AVAILABLE = False
    print("⚠️ ROUGE/BLEU libraries not available. Using implementations from scratch.")

print("✅ Environment ready!")


## 9.1 ROUGE Score (Summarization)

### Cell Purpose: Implement ROUGE metrics from scratch


In [ ]:
class ROUGEScore:
    """ROUGE (Recall-Oriented Understudy for Gisting Evaluation)"""
    
    @staticmethod
    def get_ngrams(text, n):
        """Extract n-grams from text"""
        words = text.lower().split()
        return [tuple(words[i:i+n]) for i in range(len(words)-n+1)]
    
    @staticmethod
    def rouge_n(candidate, reference, n=1):
        """
        ROUGE-N: N-gram overlap
        
        Args:
            candidate: Generated summary
            reference: Gold standard summary
            n: N-gram size (1=unigram, 2=bigram, etc.)
        """
        ref_ngrams = ROUGEScore.get_ngrams(reference, n)
        cand_ngrams = ROUGEScore.get_ngrams(candidate, n)
        
        if not ref_ngrams:
            return 0.0
        
        # Count overlapping n-grams
        ref_counter = Counter(ref_ngrams)
        cand_counter = Counter(cand_ngrams)
        
        overlap = sum((ref_counter & cand_counter).values())
        
        # ROUGE = overlap / total reference n-grams
        recall = overlap / len(ref_ngrams) if ref_ngrams else 0
        precision = overlap / len(cand_ngrams) if cand_ngrams else 0
        
        if recall + precision == 0:
            f1 = 0
        else:
            f1 = 2 * (precision * recall) / (precision + recall)
        
        return {
            'precision': precision,
            'recall': recall,
            'f1': f1
        }
    
    @staticmethod
    def rouge_l(candidate, reference):
        """
        ROUGE-L: Longest Common Subsequence
        """
        ref_words = reference.lower().split()
        cand_words = candidate.lower().split()
        
        # LCS using dynamic programming
        m, n = len(ref_words), len(cand_words)
        dp = [[0] * (n+1) for _ in range(m+1)]
        
        for i in range(1, m+1):
            for j in range(1, n+1):
                if ref_words[i-1] == cand_words[j-1]:
                    dp[i][j] = dp[i-1][j-1] + 1
                else:
                    dp[i][j] = max(dp[i-1][j], dp[i][j-1])
        
        lcs_length = dp[m][n]
        
        recall = lcs_length / m if m > 0 else 0
        precision = lcs_length / n if n > 0 else 0
        
        if recall + precision == 0:
            f1 = 0
        else:
            f1 = 2 * (precision * recall) / (precision + recall)
        
        return {
            'precision': precision,
            'recall': recall,
            'f1': f1
        }

# Example evaluation
print("="*60)
print("ROUGE EVALUATION EXAMPLE")
print("="*60)

reference = "Machine learning is a subset of artificial intelligence"
candidate1 = "Machine learning is part of AI"
candidate2 = "ML is related to artificial intelligence"

for i, candidate in enumerate([candidate1, candidate2], 1):
    print(f"\nCandidate {i}: {candidate}")
    
    rouge1 = ROUGEScore.rouge_n(candidate, reference, n=1)
    rouge2 = ROUGEScore.rouge_n(candidate, reference, n=2)
    rougel = ROUGEScore.rouge_l(candidate, reference)
    
    print(f"  ROUGE-1 F1: {rouge1['f1']:.4f}")
    print(f"  ROUGE-2 F1: {rouge2['f1']:.4f}")
    print(f"  ROUGE-L F1: {rougel['f1']:.4f}")

print("\n" + "="*60)


## 9.2 BLEU Score (Translation)

### Cell Purpose: Implement BLEU metric from scratch


In [ ]:
class BLEUScore:
    """BLEU (Bilingual Evaluation Understudy)"""
    
    @staticmethod
    def get_ngrams(text, n):
        """Extract n-grams"""
        words = text.split()
        return [tuple(words[i:i+n]) for i in range(len(words)-n+1)]
    
    @staticmethod
    def modified_precision(candidate, reference, n):
        """Calculate modified n-gram precision"""
        cand_ngrams = BLEUScore.get_ngrams(candidate, n)
        ref_ngrams = BLEUScore.get_ngrams(reference, n)
        
        if not cand_ngrams:
            return 0.0
        
        ref_counter = Counter(ref_ngrams)
        cand_counter = Counter(cand_ngrams)
        
        # Clipped counts
        clipped_counts = sum(
            min(cand_counter[ng], ref_counter[ng])
            for ng in cand_counter
        )
        
        return clipped_counts / len(cand_ngrams)
    
    @staticmethod
    def brevity_penalty(candidate, reference):
        """Calculate brevity penalty"""
        c = len(candidate.split())
        r = len(reference.split())
        
        if c > r:
            return 1.0
        else:
            return math.exp(1 - r/c) if c > 0 else 0.0
    
    @staticmethod
    def calculate(candidate, reference, max_n=4, weights=None):
        """
        Calculate BLEU score
        
        Args:
            candidate: Generated translation
            reference: Gold standard translation
            max_n: Maximum n-gram size (default: 4)
            weights: Weights for each n-gram (default: uniform)
        """
        if weights is None:
            weights = [1.0/max_n] * max_n
        
        # Calculate modified precision for each n-gram
        precisions = []
        for n in range(1, max_n+1):
            p = BLEUScore.modified_precision(candidate, reference, n)
            precisions.append(p)
        
        # Geometric mean of precisions
        if 0 in precisions:
            geo_mean = 0.0
        else:
            log_sum = sum(w * math.log(p) for w, p in zip(weights, precisions))
            geo_mean = math.exp(log_sum)
        
        # Apply brevity penalty
        bp = BLEUScore.brevity_penalty(candidate, reference)
        
        return bp * geo_mean

# Example evaluation
print("="*60)
print("BLEU EVALUATION EXAMPLE")
print("="*60)

reference = "the cat is on the mat"
candidates = [
    "the cat is on the mat",  # Perfect match
    "the cat sits on the mat",  # Close match
    "a cat is on a mat",  # Different articles
    "cat mat on",  # Poor match
]

for i, candidate in enumerate(candidates, 1):
    bleu = BLEUScore.calculate(candidate, reference)
    print(f"\nCandidate {i}: {candidate}")
    print(f"  BLEU Score: {bleu:.4f}")

print("\n" + "="*60)


## 9.3 Perplexity (Language Modeling)

### Cell Purpose: Calculate perplexity for language models


In [ ]:
class PerplexityEvaluator:
    """Perplexity: How well model predicts a sample"""
    
    @staticmethod
    def calculate(model, text_tokens, device):
        """
        Calculate perplexity
        
        Perplexity = exp(average negative log-likelihood)
        Lower perplexity = better model
        
        Args:
            model: Language model
            text_tokens: Tokenized text
            device: torch device
        """
        model.eval()
        total_loss = 0
        total_tokens = 0
        
        with torch.no_grad():
            # Process in chunks
            for i in range(len(text_tokens) - 1):
                input_ids = torch.tensor([text_tokens[i:i+1]]).to(device)
                target_ids = torch.tensor([text_tokens[i+1]]).to(device)
                
                # Get model predictions
                logits = model(input_ids)
                
                # Calculate loss
                loss = F.cross_entropy(
                    logits.view(-1, logits.size(-1)),
                    target_ids.view(-1),
                    reduction='sum'
                )
                
                total_loss += loss.item()
                total_tokens += 1
        
        # Perplexity = exp(average loss)
        avg_loss = total_loss / total_tokens if total_tokens > 0 else 0
        perplexity = math.exp(avg_loss)
        
        return perplexity
    
    @staticmethod
    def calculate_batch(model, dataloader, device):
        """Calculate perplexity over a dataset"""
        model.eval()
        total_loss = 0
        total_tokens = 0
        
        with torch.no_grad():
            for batch_x, batch_y in dataloader:
                batch_x = batch_x.to(device)
                batch_y = batch_y.to(device)
                
                logits = model(batch_x)
                loss = F.cross_entropy(
                    logits.view(-1, logits.size(-1)),
                    batch_y.view(-1),
                    reduction='sum'
                )
                
                total_loss += loss.item()
                total_tokens += batch_y.numel()
        
        avg_loss = total_loss / total_tokens if total_tokens > 0 else 0
        perplexity = math.exp(avg_loss)
        
        return perplexity, avg_loss

# Example: Compare model perplexities
print("="*60)
print("PERPLEXITY COMPARISON")
print("="*60)

# Simulate perplexity scores for different models
models_ppl = {
    "Random Baseline": 50257,  # Vocab size (uniform distribution)
    "Bigram Model": 1200,
    "Small GPT": 150,
    "Medium GPT": 45,
    "Large GPT": 20,
    "GPT-3": 15
}

plt.figure(figsize=(10, 6))
names = list(models_ppl.keys())
ppls = list(models_ppl.values())

plt.barh(names, ppls, color='#3498db', edgecolor='black', alpha=0.7)
plt.xlabel('Perplexity (lower is better)', fontsize=12, fontweight='bold')
plt.ylabel('Model', fontsize=12, fontweight='bold')
plt.title('Model Perplexity Comparison', fontsize=14, fontweight='bold')
plt.xscale('log')
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

print("\n📊 Interpretation:")
print("   - Lower perplexity = better predictions")
print("   - Perplexity ~= vocabulary size means random guessing")
print("   - Good models: perplexity 10-50")
print("   - Excellent models: perplexity <20")
print("="*60)


## 9.4 Comprehensive Evaluation Dashboard

### Cell Purpose: Create evaluation dashboard for model comparison


In [ ]:
class EvaluationDashboard:
    """Comprehensive evaluation dashboard"""
    
    def __init__(self):
        self.results = {}
    
    def add_result(self, model_name, task, metric_name, score):
        """Add evaluation result"""
        if model_name not in self.results:
            self.results[model_name] = {}
        if task not in self.results[model_name]:
            self.results[model_name][task] = {}
        
        self.results[model_name][task][metric_name] = score
    
    def compare_models(self, task):
        """Compare models on a specific task"""
        print("="*60)
        print(f"MODEL COMPARISON: {task.upper()}")
        print("="*60)
        
        models = list(self.results.keys())
        metrics = list(self.results[models[0]][task].keys())
        
        # Print table
        header = f"{'Model':<20}"
        for metric in metrics:
            header += f"{metric:<15}"
        print(header)
        print("-"*60)
        
        for model in models:
            row = f"{model:<20}"
            for metric in metrics:
                score = self.results[model][task][metric]
                row += f"{score:<15.4f}"
            print(row)
        
        print("="*60)
    
    def visualize_all(self):
        """Create comprehensive visualization"""
        models = list(self.results.keys())
        tasks = list(self.results[models[0]].keys())
        
        fig, axes = plt.subplots(1, len(tasks), figsize=(5*len(tasks), 5))
        if len(tasks) == 1:
            axes = [axes]
        
        for idx, task in enumerate(tasks):
            ax = axes[idx]
            
            # Get scores for this task
            metrics = list(self.results[models[0]][task].keys())
            x = np.arange(len(models))
            width = 0.8 / len(metrics)
            
            for i, metric in enumerate(metrics):
                scores = [self.results[m][task][metric] for m in models]
                offset = (i - len(metrics)/2) * width
                ax.bar(x + offset, scores, width, label=metric, alpha=0.7)
            
            ax.set_xlabel('Model', fontweight='bold')
            ax.set_ylabel('Score', fontweight='bold')
            ax.set_title(f'{task.title()} Performance', fontweight='bold')
            ax.set_xticks(x)
            ax.set_xticklabels(models, rotation=45, ha='right')
            ax.legend()
            ax.grid(axis='y', alpha=0.3)
        
        plt.tight_layout()
        plt.show()

# Demo dashboard
dashboard = EvaluationDashboard()

# Add sample results
dashboard.add_result("Model A", "summarization", "ROUGE-1", 0.45)
dashboard.add_result("Model A", "summarization", "ROUGE-2", 0.22)
dashboard.add_result("Model A", "translation", "BLEU", 0.38)

dashboard.add_result("Model B", "summarization", "ROUGE-1", 0.52)
dashboard.add_result("Model B", "summarization", "ROUGE-2", 0.28)
dashboard.add_result("Model B", "translation", "BLEU", 0.42)

# Compare
dashboard.compare_models("summarization")
dashboard.visualize_all()

print("✅ Evaluation dashboard ready!")


## 9.5 Production Monitoring

### Cell Purpose: AWS CloudWatch integration for continuous evaluation


In [ ]:
monitoring_guide = """
═══════════════════════════════════════════════════════════
PRODUCTION MONITORING & EVALUATION
═══════════════════════════════════════════════════════════

## Real-Time Metrics

### 1. Model Performance Metrics
--------------------------------------------------------------
- **Perplexity**: Track on holdout set daily
- **ROUGE/BLEU**: Sample evaluation on recent predictions
- **Latency**: P50, P95, P99 response times
- **Throughput**: Requests per second

### 2. Data Quality Metrics
--------------------------------------------------------------
- **Input Length Distribution**: Detect unusual patterns
- **Output Length Distribution**: Check for degeneration
- **Token Distribution**: Monitor vocabulary usage
- **Error Rates**: Track generation failures

## AWS CloudWatch Implementation

```python
import boto3
from datetime import datetime

class ModelMonitor:
    def __init__(self, model_name):
        self.model_name = model_name
        self.cloudwatch = boto3.client('cloudwatch')
    
    def log_metric(self, metric_name, value, unit='None'):
        self.cloudwatch.put_metric_data(
            Namespace='LLM/Evaluation',
            MetricData=[{
                'MetricName': metric_name,
                'Value': value,
                'Unit': unit,
                'Timestamp': datetime.utcnow(),
                'Dimensions': [
                    {'Name': 'ModelName', 'Value': self.model_name}
                ]
            }]
        )
    
    def log_inference(self, latency_ms, tokens_generated, success=True):
        self.log_metric('InferenceLatency', latency_ms, 'Milliseconds')
        self.log_metric('TokensGenerated', tokens_generated, 'Count')
        self.log_metric('InferenceSuccess', 1 if success else 0, 'Count')
    
    def log_quality(self, rouge1, rouge2, bleu):
        self.log_metric('ROUGE-1', rouge1, 'None')
        self.log_metric('ROUGE-2', rouge2, 'None')
        self.log_metric('BLEU', bleu, 'None')
```

### 3. Create CloudWatch Dashboard
--------------------------------------------------------------
```python
# dashboard_config.json
{
  "widgets": [
    {
      "type": "metric",
      "properties": {
        "metrics": [
          ["LLM/Evaluation", "ROUGE-1", {"stat": "Average"}],
          [".", "ROUGE-2", {"stat": "Average"}],
          [".", "BLEU", {"stat": "Average"}]
        ],
        "period": 300,
        "stat": "Average",
        "region": "us-east-1",
        "title": "Model Quality Metrics"
      }
    },
    {
      "type": "metric",
      "properties": {
        "metrics": [
          ["LLM/Evaluation", "InferenceLatency", {"stat": "Average"}]
        ],
        "period": 60,
        "stat": "Average",
        "region": "us-east-1",
        "title": "Inference Latency (ms)"
      }
    }
  ]
}
```

## Automated Quality Checks

### Daily Evaluation Script
--------------------------------------------------------------
```python
# daily_eval.py
import schedule
import time

def daily_evaluation():
    # Load test set
    test_set = load_test_data()
    
    # Evaluate model
    rouge_scores = evaluate_rouge(model, test_set)
    bleu_scores = evaluate_bleu(model, test_set)
    perplexity = calculate_perplexity(model, test_set)
    
    # Log to CloudWatch
    monitor = ModelMonitor('production-model')
    monitor.log_metric('DailyRouge1', rouge_scores['rouge1'])
    monitor.log_metric('DailyBLEU', bleu_scores)
    monitor.log_metric('DailyPerplexity', perplexity)
    
    # Alert if metrics degrade
    if rouge_scores['rouge1'] < THRESHOLD:
        send_alert("Model quality degraded!")

# Schedule daily
schedule.every().day.at("02:00").do(daily_evaluation)

while True:
    schedule.run_pending()
    time.sleep(3600)
```

## Alerting Rules

### CloudWatch Alarms
--------------------------------------------------------------
```python
cloudwatch = boto3.client('cloudwatch')

# Alert if ROUGE-1 drops below threshold
cloudwatch.put_metric_alarm(
    AlarmName='LLM-Quality-Drop',
    MetricName='ROUGE-1',
    Namespace='LLM/Evaluation',
    Statistic='Average',
    Period=3600,
    EvaluationPeriods=2,
    Threshold=0.40,
    ComparisonOperator='LessThanThreshold',
    AlarmActions=['arn:aws:sns:us-east-1:123456789:alerts']
)

# Alert if latency exceeds threshold
cloudwatch.put_metric_alarm(
    AlarmName='LLM-High-Latency',
    MetricName='InferenceLatency',
    Namespace='LLM/Evaluation',
    Statistic='Average',
    Period=300,
    EvaluationPeriods=1,
    Threshold=500,  # 500ms
    ComparisonOperator='GreaterThanThreshold'
)
```

## Cost for Monitoring
--------------------------------------------------------------
- CloudWatch metrics: $0.30 per metric/month
- CloudWatch dashboards: $3.00/month
- CloudWatch alarms: $0.10 per alarm/month
- Total: ~$5-10/month for comprehensive monitoring

═══════════════════════════════════════════════════════════
"""

print(monitoring_guide)


## 📝 Chapter Summary

### What We Built:
1. ✅ **ROUGE Score**: Summarization evaluation from scratch
2. ✅ **BLEU Score**: Translation quality metric
3. ✅ **Perplexity**: Language modeling capability
4. ✅ **Evaluation Dashboard**: Compare multiple models
5. ✅ **Production Monitoring**: CloudWatch integration
6. ✅ **Automated Alerts**: Quality degradation detection

### Metrics Overview:

| Metric | Use Case | Range | Better Score |
|--------|----------|-------|--------------|
| ROUGE-1 | Summarization | 0-1 | Higher |
| ROUGE-2 | Summarization | 0-1 | Higher |
| ROUGE-L | Summarization | 0-1 | Higher |
| BLEU | Translation | 0-1 | Higher |
| Perplexity | Language Model | 1-∞ | Lower |

### When to Use Each Metric:

**ROUGE (Recall-Oriented)**
- Best for: Summarization
- Measures: N-gram overlap with reference
- Variants: ROUGE-1 (unigrams), ROUGE-2 (bigrams), ROUGE-L (LCS)

**BLEU (Precision-Oriented)**
- Best for: Translation
- Measures: Modified n-gram precision
- Note: Requires exact word matches

**Perplexity**
- Best for: Language modeling
- Measures: How "surprised" model is by text
- Lower = better predictions

**Human Evaluation**
- Gold standard for quality
- Expensive and time-consuming
- Use for final validation

### Production Monitoring Strategy:

1. **Continuous Metrics**:
   - Latency (every request)
   - Throughput (every minute)
   - Error rates (every request)

2. **Daily Batch Evaluation**:
   - ROUGE/BLEU on sample (100-1000 examples)
   - Perplexity on validation set
   - Distribution checks

3. **Weekly Deep Dive**:
   - Manual inspection of outputs
   - Edge case testing
   - Bias/toxicity checks

4. **Monthly Reviews**:
   - Full test set evaluation
   - Human evaluation sample
   - Model comparison

### Cost Summary:
- **Metrics Computation**: Free (run locally or on training instances)
- **CloudWatch Monitoring**: ~$5-10/month
- **Automated Evaluation**: ~$2-5/month (spot instances)
- **Total**: ~$7-15/month for comprehensive monitoring

### Best Practices:
1. **Multiple Metrics**: No single metric tells the full story
2. **Task-Specific**: Choose metrics appropriate for your task
3. **Baseline**: Always compare against baseline models
4. **Trends**: Monitor changes over time, not absolute values
5. **Human Eval**: Sample-based human evaluation for final validation

### Next Steps:
➡️ **Advanced Evaluation**: BERTScore, METEOR, CIDEr  
➡️ **Bias Detection**: Fairness and toxicity metrics  
➡️ **User Feedback**: Incorporate real user ratings  
➡️ **Continuous Improvement**: Retrain based on metrics  

---

## 🔗 Resources

**Papers:**
- [ROUGE Paper](https://aclanthology.org/W04-1013/)
- [BLEU Paper](https://aclanthology.org/P02-1040/)
- [BERTScore](https://arxiv.org/abs/1904.09675)
- [Beyond Accuracy](https://arxiv.org/abs/2202.00666) - Holistic evaluation

**Tools:**
- [rouge-score](https://github.com/google-research/google-research/tree/master/rouge) - Google's implementation
- [sacrebleu](https://github.com/mjpost/sacrebleu) - Standard BLEU
- [nlg-eval](https://github.com/Maluuba/nlg-eval) - Multiple metrics
- [lm-evaluation-harness](https://github.com/EleutherAI/lm-evaluation-harness) - Comprehensive suite

**AWS Documentation:**
- [CloudWatch Custom Metrics](https://docs.aws.amazon.com/AmazonCloudWatch/latest/monitoring/publishingMetrics.html)
- [CloudWatch Dashboards](https://docs.aws.amazon.com/AmazonCloudWatch/latest/monitoring/CloudWatch_Dashboards.html)
- [CloudWatch Alarms](https://docs.aws.amazon.com/AmazonCloudWatch/latest/monitoring/AlarmThatSendsEmail.html)

**🎉 Congratulations! You've completed the complete LLM series from fundamentals to production! 🎉**
